Applicative Functors in Python
---
While we see implementations of the Functor and Monad in Python, we rarely see an implementation of the Applicative Functor in Python. I feel that the reason is to do with the examples usually given as motivations for the use of applicatives in text on functional programming. Take as an example [Functional Pearls](http://www.cs.umd.edu/class/spring2019/cmsc388F/lectures/applicative-functors.html) from the University of Maryland. They give as motivation an extended version of the motivation Graham Hutton uses in programming in Haskell, that of nondetermistic computation. The motivation adapted to Python (for the Haskell version see Functional Pearls) goes like this: 

Consider a list of numbers as nondeterministic. That is if `x = [1,2,3,4,5]` x could be one of those numbers. What if we want to write a function that takes a function, which takes two arguments, and two seperate list of arguments to get one list of arguments back. See the function signature of such a function.

In [9]:
from typing import Callable, TypeVar, Any  

S = TypeVar('S')
T = TypeVar('T')
U = TypeVar('U')


def lift2(func: Callable[[S],U], xs: list[S], ys: list[T]) -> list[U]:
    ...

We could this computation with an applicative list. Let's define one:

In [3]:
from abc import ABC, abstractmethod

class Functor(ABC):
    @abstractmethod
    def fmap(self, func):
        pass

class Applicative(Functor):
    @abstractmethod
    def apply(self, func_applicative):
        pass
    
    @staticmethod
    def pure(value):
        pass

# A concrete implementation of an Applicative for a List-like context
class ListApplicative(Applicative):
    def __init__(self, values):
        self.values = values

    def fmap(self, func):
        return ListApplicative([func(x) for x in self.values])

    def apply(self, func_applicative):
        # func_applicative is expected to be another ListApplicative containing functions
        return ListApplicative([f(x) for f in func_applicative.values for x in self.values])

    @staticmethod
    def pure(value):
        return ListApplicative([value])

    def __repr__(self):
        return f'ListApplicative({self.values})'

An applicative is intermediary structure in between the functor and the monad. We can see that from 
our definition. An Appliactive subclasses the Functor. Furthermore any applicative structure must implement to functions. 

1. pure
2. apply

The ListApplicative thus creates at minimal three functions, fmap, pure, and apply. Now how can we do non deterministic computation with that?

In [6]:
import functools

def curried_add(n: int) -> Callable[[U],T]:
    def inner(m: int) -> int:
        return n + m 
    return inner

x = ListApplicative([1,2,3,4,5])
y = ListApplicative([101,102,103,104,105])

def lift2(func: Callable[[S],U], xs: list[S], ys: list[T]) -> list[U]:
    partial_f = functools.partial(func)
    return y.apply(x.apply(ListApplicative.pure(partial_f)))

lift2(curried_add, x, y)

ListApplicative([102, 103, 104, 105, 106, 103, 104, 105, 106, 107, 104, 105, 106, 107, 108, 105, 106, 107, 108, 109, 106, 107, 108, 109, 110])

Now if you do not find this very motivating I can understand that, neither do I. It seems to me that 
we can easily achieve this with a list comprehension, in both languages. 

In [8]:
from operator import add 

def liftPythonic(func: Callable[[S],U], xs: list[S], ys: list[T]) -> list[U]:
    return [func(x,y) for x in xs for y in ys]

liftPythonic(add, [1,2,3,4,5], [101,102,103,104,105])

[102,
 103,
 104,
 105,
 106,
 103,
 104,
 105,
 106,
 107,
 104,
 105,
 106,
 107,
 108,
 105,
 106,
 107,
 108,
 109,
 106,
 107,
 108,
 109,
 110]

If we want to an operation a iterable, a generator or comprehension would be preferable. 
I just point out that apply uses a list comprehension in its body. Applying it to two list is just composition. Which raises the question what could function as a motivation for programming applicative structures. 

#### Motivating the Applicative
---
The main motivation for using an applicative is threefold:

1. You are working with multiple functor structures like; list, options, futures, etc.
2. You want to abstract over side effects or contexts in a reusable way.
3. You need to handle validation, optional values, parallel computations, or other effects.

In this artikel will give an example of all three. 

#### Applicative Functors Generalize Beyond Lists

While list comprehensions are great for simple lists, Applicative functors provide a general interface that works with any context, not just lists. This could include:

- Optional values (Maybe or Option in other languages): Handling computations that might fail.
- Concurrent computations (Future, Async): Handling computations that may not return a result immediately.
- Validation: Combining multiple validation checks that could fail independently.
- Monads like Either, Result, etc.

Consider the situation where you have optional values and you only want to a function to be computed if we have values for both options. With an applicative we could lift the function to the Optional context and apply the function to both values. 

In [29]:
from typing import Callable, Generic, TypeVar, Optional

# Define type variables
S = TypeVar('S')  # Input type
T = TypeVar('T')  # Output type

class OptionApplicative(Generic[S]):
    def __init__(self, value: Optional[S]):
        self.value = value

    # fmap method with type annotations
    def fmap(self, func: Callable[[S], T]) -> 'OptionApplicative[T]':
        if self.value is None:
            return OptionApplicative(None)
        return OptionApplicative(func(self.value))

    # apply method with type annotations
    def apply(self, func_applicative: 'OptionApplicative[Callable[[S], T]]') -> 'OptionApplicative[T]':
        if self.value is None or func_applicative.value is None:
            return OptionApplicative(None)
        return OptionApplicative(func_applicative.value(self.value))

    @staticmethod
    def pure(value: S) -> 'OptionApplicative[S]':
        return OptionApplicative(value)

    def __repr__(self) -> str:
        return f'OptionApplicative({self.value})'

In [30]:
def lift2(func, op1: OptionApplicative[S], op2: OptionApplicative[S]) -> OptionApplicative[T]: 
    '''needs a curried function'''
    pfunc = functools.partial(func)
    op2.apply(op1.apply(OptionApplicative.pure(pfunc)))

op1 = OptionApplicative(19)
op2 = OptionApplicative(23)
print(lift2(curried_add, op1, op2))

None


In [35]:
def succ(n: int) -> int:
    return n + 1 

op1.apply(OptionApplicative.pure(succ))

OptionApplicative(20)